In [2]:
from nanome.service import PluginService
from nanome.interface import PluginInstanceRedisInterface

# Set up redis credentials
redis_host = 'redis'
redis_port = 6379
redis_password = ''

# When your PluginService is running, you can get your channel value from the Logs, or from the query parameter in your open browser.
# Update this value to match that, so that your commands will run against your live workspace.
redis_channel = 'a5100bcd-3b33-48dc-a8b0-67f88182f24e'

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
print('Hello')

Hello


In [3]:
# Test connection to Session
comps = plugin_instance.request_complex_list()
print(comps)

Sending request_complex_list Request to Redis Channel a5100bcd-3b33-48dc-a8b0-67f88182f24e
Response received on channel 9109f5dc-5af2-459a-805e-b01417ceb3e1
[]


In [4]:
# neo4j setup
from py2neo import Graph, Node, Relationship

neo4j_uri = 'bolt://neo4j:7687/'
username = 'neo4j'
password = 'F8dwclytew'
graph = Graph(neo4j_uri, auth=(username, password))


In [5]:
# Create neo4j database
# with open("assets/neo4j_movie_graph_query.txt") as f:
#     query_string = f.read()
# results = graph.run(query_string)
# data = results.data()

In [6]:
from nanome.api import structure
from nanome.util import Vector3, Color

def create_node_complex(node, atom_position):
    comp = structure.Complex()
    mol = structure.Molecule()
    chain = structure.Chain()
    res = structure.Residue()
    atom = structure.Atom()

    atom.position = atom_position
    atom.labeled = True

    label_text = ''
    if node.labels.__str__() == ':Movie':
        # Movie node
        label_text = node['title']
        atom.atom_color = Color(0, 255, 0)
    else:
        # Person Node
        label_text = node['name']
        atom.atom_color = Color(0, 0, 255)
    atom.label_text = label_text
    res.add_atom(atom)
    chain.add_residue(res)
    mol.add_chain(chain)
    comp.add_molecule(mol)
    comp.name = atom.label_text
    return comp


In [7]:
import nanome
from nanome.api.shapes import Label, Line, Shape

def draw_relationship_line(struct1, struct2):
    """Draw line connecting two structs.

    :arg struct1: struct
    :arg struct2: struct
    :arg line_settings: Dict describing shape and color of line based on interaction_type
    """
    line = Line()
    for struct, anchor in zip([struct1, struct2], line.anchors):
        anchor.anchor_type = nanome.util.enums.ShapeAnchorType.Atom
        anchor.target = next(struct.atoms).index
    return line
    

In [8]:
import random

def get_or_create_comp_node(node, comp_nodes, position):
    label_text = ''
    created = False
    if node.labels.__str__() == ':Movie':
        # Movie node
        label_text = node['title']
    else:
        # Person Node
        label_text = node['name']
    if label_text not in [comp.name for comp in comp_nodes]:
        comp_node = create_node_complex(node, position)
        comp_node = plugin_instance.add_to_workspace([comp_node])[0]
        comp_nodes.append(comp_node)
        created = True
    else:
        comp_node = next(iter(comp for comp in comp_nodes if comp.name == label_text))
    return comp_node, created

def render_results(path_results):
#     position = Vector3(15, 15, 15)
    comp_nodes = []

    for record in path_results:
        path = record['p']
        rels = path.relationships
        path_nodes = []
        for rel in rels:
            start_node = rel.start_node
            end_node = rel.end_node
            min_val = 40
            max_val = 80
            position = Vector3(random.randrange(min_val, max_val), random.randrange(min_val, max_val), random.randrange(min_val, max_val))
            start_comp_node, created = get_or_create_comp_node(start_node, comp_nodes, position)
            position = Vector3(random.randrange(min_val, max_val), random.randrange(min_val, max_val), random.randrange(min_val, max_val))
            end_comp_node, created = get_or_create_comp_node(end_node, comp_nodes, position)
        line = draw_relationship_line(start_comp_node, end_comp_node)
        plugin_instance.upload_shapes([line])
    return comp_nodes


In [9]:
from nanome.api.structure import Workspace

ws = Workspace()
plugin_instance.update_workspace(ws)
result = graph.run("MATCH p=()-[r:ACTED_IN]->() RETURN p LIMIT 25")
node_list = render_results(result)


Sending update_workspace Request to Redis Channel a5100bcd-3b33-48dc-a8b0-67f88182f24e
Response received on channel 52136b34-ab5b-49ca-868d-8f2d73d70e2d
57.0 43.0 48.0
Sending add_to_workspace Request to Redis Channel a5100bcd-3b33-48dc-a8b0-67f88182f24e
Response received on channel 782b3192-54d3-4fa4-9965-64ef66e86484
52.0 77.0 56.0
Sending add_to_workspace Request to Redis Channel a5100bcd-3b33-48dc-a8b0-67f88182f24e
Response received on channel feeada59-a9bc-4ed5-983a-32951475623f
Sending upload_shapes Request to Redis Channel a5100bcd-3b33-48dc-a8b0-67f88182f24e
Response received on channel 314bf7da-5e73-437e-91aa-d3691dad4183
68.0 65.0 54.0
51.0 48.0 64.0
Sending add_to_workspace Request to Redis Channel a5100bcd-3b33-48dc-a8b0-67f88182f24e
Response received on channel 0601268c-40cc-4d65-93c2-6d3b7687e7e1
Sending upload_shapes Request to Redis Channel a5100bcd-3b33-48dc-a8b0-67f88182f24e
Response received on channel acb6d57a-e970-4ffe-816e-8acc583b6e4d
40.0 61.0 66.0
60.0 76.0 44.